In [1]:
from pprint import pprint
from dataclasses import asdict
import pandas as pd
import json
import os

from martignac.nomad.utils import get_nomad_request, post_nomad_request
from martignac.nomad.entries import get_entry_by_id, query_entries
from martignac.nomad.datasets import retrieve_datasets, create_dataset, delete_dataset
from martignac.nomad.uploads import upload_files_to_nomad, get_upload_by_id, publish_upload, edit_upload_metadata
from martignac.nomad.users import get_user_by_id
from martignac.nomad.queries import query_entries_in_mini_format, find_mini_queries_corresponding_to_job, find_mini_queries_corresponding_to_workflow
from martignac.workflows.solute_in_solvent_alchemical import SoluteInSolventAlchemicalFlow

24-08-22 09:08:03 - MDAnalysis.coordinates.AMBER - WARNING - netCDF4 is not available. Writing AMBER ncdf files will be slow.
24-08-22 09:08:03 - pymbar.timeseries - WARNING - Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
24-08-22 09:08:03 - pymbar.mbar_solvers - WARNING - 
********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************
24-08-22 09:08:03 - pymbar.mbar_solvers - INFO - JAX was either not detected or disabled, using standard NumPy and SciPy


In [6]:
df = pd.json_normalize([asdict(e) for e in find_mini_queries_corresponding_to_workflow(SoluteInSolventAlchemicalFlow)])
df = df.sort_values(["comment.state_point.solvent_name", "comment.state_point.lambda_state"], ignore_index=True)
df.head(5)

24-08-22 09:11:41 - martignac.nomad.queries - INFO - querying comments on test server
24-08-22 09:11:41 - martignac.nomad.queries - INFO - querying entries with query {'query': {'authors.name': {'any': ['Tristan Bereau', 'Joseph Rudzinski']}, 'entry_type': 'Workflow'}, 'pagination': {'page_size': 25}, 'required': {'include': ['entry_id', 'upload_id', 'comment', 'published']}}
24-08-22 09:11:41 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 09:11:42 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 09:11:42 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 09:11:42 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query
24-08-22 09:11:43 - martignac.nomad.queries - INFO - found 82 Martignac entries online
24-08-22 09:11:43 

,entry_id,upload_id,published,workflow_name,use_prod,comment.job_id,comment.workflow_name,comment.state_point.type,comment.state_point.solvent_name,comment.state_point.solute_name,comment.state_point.lambda_state,comment.mdp_files,comment.itp_files
0,CPo3byidQdHyFlm6LH2MXrW-LYCi,A74Kxh-dRx-376A9yuXDIg,True,SoluteInSolventAlchemicalFlow,False,e30741699db6e4a0f8c104fc0f7feb59,SoluteInSolventAlchemicalFlow,solute_in_solvent_alchemical,CHEX,P6,0,89b51ecb08ca7fb84bfa3118d1ede5b6,15259a565eb93668e9e40eb8cfd10f9b
1,S4CkHz37dr-6VqqC01pm0D1LMTON,A74Kxh-dRx-376A9yuXDIg,True,SoluteInSolventAlchemicalFlow,False,e30741699db6e4a0f8c104fc0f7feb59,SoluteInSolventAlchemicalFlow,solute_in_solvent_alchemical,CHEX,P6,0,89b51ecb08ca7fb84bfa3118d1ede5b6,15259a565eb93668e9e40eb8cfd10f9b
2,3y8AOAVa1ruV1QlUAuQHl3eaELak,A74Kxh-dRx-376A9yuXDIg,True,SoluteInSolventAlchemicalFlow,False,01f00beb359b58fe953f8461101a34cc,SoluteInSolventAlchemicalFlow,solute_in_solvent_alchemical,CHEX,P6,1,89b51ecb08ca7fb84bfa3118d1ede5b6,15259a565eb93668e9e40eb8cfd10f9b
3,HIiqLBlZd1WpyDGcs9uiF8szuZXF,A74Kxh-dRx-376A9yuXDIg,True,SoluteInSolventAlchemicalFlow,False,4ced8314d8d94472cf6459ce33c87b17,SoluteInSolventAlchemicalFlow,solute_in_solvent_alchemical,CHEX,P6,2,89b51ecb08ca7fb84bfa3118d1ede5b6,15259a565eb93668e9e40eb8cfd10f9b
4,NjRfbhXfDBze5qYX_poTJ_z10QoY,A74Kxh-dRx-376A9yuXDIg,True,SoluteInSolventAlchemicalFlow,False,a31c784e5db3612b3b73475b1c8d0832,SoluteInSolventAlchemicalFlow,solute_in_solvent_alchemical,CHEX,P6,3,89b51ecb08ca7fb84bfa3118d1ede5b6,15259a565eb93668e9e40eb8cfd10f9b


In [3]:
nomad_user_me = get_user_by_id("30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b")
nomad_user_me

24-08-20 19:43:33 - martignac.nomad.users - INFO - retrieving user 30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b on prod server
24-08-20 19:43:33 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/users/30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b


NomadUser(name='Tristan Bereau')

In [2]:
upload_id = upload_files_to_nomad('./martini_fep/workspace/008bec4a5b5a9ef9dc344f7105eaf20f.zip')
upload_id

24-02-14 10:43:55 - martignac.nomad.uploads - INFO - uploading file ./martini_fep/workspace/008bec4a5b5a9ef9dc344f7105eaf20f.zip on test server
24-02-14 10:43:55 - martignac.nomad.utils - INFO - Requesting authentication token @ https://nomad-lab.eu/prod/v1/test/api/v1
24-02-14 10:43:56 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads
24-02-14 10:43:56 - martignac.nomad.uploads - INFO - successful upload to Hbwh9S0-RLehQIh90nfhAQ


'Hbwh9S0-RLehQIh90nfhAQ'

In [3]:
nomad_upload = get_upload_by_id("Hbwh9S0-RLehQIh90nfhAQ")
pprint(nomad_upload)

24-08-16 15:12:16 - martignac.nomad.uploads - INFO - retrieving upload Hbwh9S0-RLehQIh90nfhAQ on test server
24-08-16 15:12:16 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/Hbwh9S0-RLehQIh90nfhAQ
24-08-16 15:12:16 - martignac.nomad.utils - INFO - Requesting authentication token @ https://nomad-lab.eu/prod/v1/test/api/v1
24-08-16 15:12:17 - martignac.nomad.users - INFO - retrieving user 30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b on prod server
24-08-16 15:12:17 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/users/30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b


NomadUpload(upload_id='Hbwh9S0-RLehQIh90nfhAQ', upload_create_time=datetime.datetime(2024, 2, 14, 9, 43, 58, 640000), main_author=NomadUser(name='Tristan Bereau'), process_running=False, current_process='edit_upload_metadata', process_status='SUCCESS', last_status_message='Process edit_upload_metadata completed successfully', errors=[], warnings=[], coauthors=[], coauthor_groups=[], reviewers=[], reviewer_groups=[], writers=[NomadUser(name='Tristan Bereau')], writer_groups=[], viewers=[NomadUser(name='Tristan Bereau')], viewer_groups=[], published=True, published_to=[], with_embargo=False, embargo_length=0.0, license='CC BY 4.0', entries=1, n_entries=None, upload_files_server_path=None, publish_time=datetime.datetime(2024, 2, 14, 12, 23, 57, 957000), references=None, datasets=None, external_db=None, upload_name=None, comment=None, use_prod=False, complete_time=datetime.datetime(2024, 2, 14, 13, 9, 50, 637000))


In [10]:
edit_upload_metadata("Hbwh9S0-RLehQIh90nfhAQ", dataset_id='HJdEI1q4SV-c5Di43BTT_Q', use_prod=False)

24-02-14 14:09:50 - martignac.nomad.uploads - INFO - editing the metadata for upload Hbwh9S0-RLehQIh90nfhAQ on test server
24-02-14 14:09:50 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/Hbwh9S0-RLehQIh90nfhAQ/edit


{'upload_id': 'Hbwh9S0-RLehQIh90nfhAQ',
 'data': {'process_running': True,
  'current_process': 'edit_upload_metadata',
  'process_status': 'PENDING',
  'last_status_message': 'Pending: edit_upload_metadata',
  'errors': [],
  'warnings': [],
  'complete_time': '2024-02-14T13:09:44.461000',
  'upload_id': 'Hbwh9S0-RLehQIh90nfhAQ',
  'upload_create_time': '2024-02-14T09:43:58.640000',
  'main_author': '30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b',
  'coauthors': [],
  'coauthor_groups': [],
  'reviewers': [],
  'reviewer_groups': [],
  'writers': ['30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b'],
  'writer_groups': [],
  'viewers': ['30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b'],
  'viewer_groups': [],
  'published': True,
  'published_to': [],
  'publish_time': '2024-02-14T12:23:57.957000',
  'with_embargo': False,
  'embargo_length': 0,
  'license': 'CC BY 4.0',
  'entries': 1}}

In [4]:
publish_upload(nomad_upload.upload_id, use_prod=False)

24-02-14 13:23:55 - martignac.nomad.uploads - INFO - publishing upload Hbwh9S0-RLehQIh90nfhAQ on test server
24-02-14 13:23:55 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/uploads/Hbwh9S0-RLehQIh90nfhAQ/action/publish


{'upload_id': 'Hbwh9S0-RLehQIh90nfhAQ',
 'data': {'process_running': True,
  'current_process': 'publish_upload',
  'process_status': 'PENDING',
  'last_status_message': 'Pending: publish_upload',
  'errors': [],
  'warnings': [],
  'complete_time': '2024-02-14T09:44:01.513000',
  'upload_id': 'Hbwh9S0-RLehQIh90nfhAQ',
  'upload_create_time': '2024-02-14T09:43:58.640000',
  'main_author': '30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b',
  'coauthors': [],
  'coauthor_groups': [],
  'reviewers': [],
  'reviewer_groups': [],
  'writers': ['30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b'],
  'writer_groups': [],
  'viewers': ['30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b'],
  'viewer_groups': [],
  'published': False,
  'published_to': [],
  'with_embargo': False,
  'embargo_length': 0,
  'license': 'CC BY 4.0',
  'entries': 1,
  'upload_files_server_path': '/nomad/test/fs/staging/H/Hbwh9S0-RLehQIh90nfhAQ'}}

In [5]:
my_datasets = retrieve_datasets(user_id=nomad_user_me.user_id, use_prod=False, max_datasets=20)
pprint(my_datasets)

24-02-14 14:16:14 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/datasets/?user_id=30d3a108-d2cc-45ec-9ddb-0c1dc6a2c99b&page_size=10


[NomadDataset(dataset_id='HJdEI1q4SV-c5Di43BTT_Q',
              dataset_create_time=datetime.datetime(2024, 2, 14, 9, 39, 55, 876000),
              dataset_name='Martignac test dataset',
              dataset_type='DatasetType.owned',
              dataset_modified_time=datetime.datetime(2024, 2, 14, 9, 39, 55, 876000),
              user=NomadUser(name='Tristan Bereau'),
              doi=None,
              pid=None)]


In [4]:
dataset_id = create_dataset("Martignac test dataset")
dataset_id

24-02-14 10:39:53 - martignac.nomad.datasets - INFO - creating dataset name Martignac test dataset on test server
24-02-14 10:39:53 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/datasets/


'HJdEI1q4SV-c5Di43BTT_Q'

In [4]:
pprint(retrieve_datasets(dataset_name="Martignac test dataset", use_prod=False, max_datasets=20))

24-02-14 11:52:50 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/datasets/?dataset_name=Martignac test dataset&page_size=10


[NomadDataset(dataset_id='HJdEI1q4SV-c5Di43BTT_Q',
              dataset_create_time=datetime.datetime(2024, 2, 14, 9, 39, 55, 876000),
              dataset_name='Martignac test dataset',
              dataset_type='DatasetType.owned',
              dataset_modified_time=datetime.datetime(2024, 2, 14, 9, 39, 55, 876000),
              user=NomadUser(name='Tristan Bereau'),
              doi=None,
              pid=None)]


In [3]:
entry = get_entry_by_id("IWfLp8VCyT7z9t3BtVy21Q5WZSRW", with_authentication=True, use_prod=False)
pprint(entry)

24-02-14 14:28:24 - martignac.nomad.utils - INFO - Sending get request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/IWfLp8VCyT7z9t3BtVy21Q5WZSRW


NomadEntry(entry_id='IWfLp8VCyT7z9t3BtVy21Q5WZSRW',
           upload_id='Hbwh9S0-RLehQIh90nfhAQ',
           references=[],
           origin='Tristan Bereau',
           n_quantities=0,
           nomad_version='1.2.2.dev365+g0c980916a',
           upload_create_time=datetime.datetime(2024, 2, 14, 9, 43, 58, 640000, tzinfo=datetime.timezone(datetime.timedelta(0), '+0000')),
           nomad_commit='',
           processing_errors=['Error creating MDAnalysis universe.',
                              'Error creating MDAnalysis universe.',
                              'Error creating MDAnalysis universe.',
                              'Error parsing interactions.',
                              'Error creating MDAnalysis universe.',
                              'Error creating MDAnalysis universe.',
                              'Error creating MDAnalysis universe.',
                              'Error creating MDAnalysis universe.',
                              'Error creating MDA

In [6]:
# delete_dataset(dataset_id='E5l1hls-S2SJObJXzmQCSQ')

In [4]:
entry = query_entries(origin="Tristan Bereau", use_prod=False)
entry

24-02-14 12:02:37 - martignac.nomad.utils - INFO - Sending post request @ https://nomad-lab.eu/prod/v1/test/api/v1/entries/query


[]

In [3]:
pprint(entry)

NomadEntry(entry_id='-0cWC-cNbauZkQeHl4rDQf7ttiEL',
           upload_id='gfMPbAADR_agE8pKdhckzQ',
           references=['https://doi.org/10.1063/5.0104914',
                       'http://doi.org/10.5281/zenodo.6032826'],
           origin='Joseph Rudzinski',
           n_quantities=352261,
           nomad_version='1.2.0rc1.dev228+gfc977c55e',
           upload_create_time=datetime.datetime(2023, 7, 4, 14, 59, 35, 670000, tzinfo=datetime.timezone(datetime.timedelta(0), '+0000')),
           nomad_commit='',
           processing_errors=[],
           entry_name='C2800H5280O800 GROMACS MolecularDynamics simulation',
           last_processing_time=datetime.datetime(2023, 7, 4, 15, 55, 4, 750000, tzinfo=datetime.timezone(datetime.timedelta(0), '+0000')),
           parser_name='parsers/gromacs',
           calc_id='-0cWC-cNbauZkQeHl4rDQf7ttiEL',
           published=True,
           writers=[NomadUser(name='Joseph Rudzinski'),
                    NomadUser(name='Tristan Bereau'),
    